In [ ]:
"""import pandas as pd
from urllib.parse import quote
import requests
import json

response_data = []
title_year_set = set()

api_key = 'b5546873'
api_url = 'http://www.omdbapi.com/?t={}&y={}&apikey={}'

budget_data = pd.read_csv('top_grossing_movies.csv')
titles_and_years = budget_data[['highest_grossing_year', 'Title']]

for index, row in titles_and_years.iterrows():
    title_year_set.add((row['Title'], row['highest_grossing_year']))

    
title_year_set = sorted(title_year_set)
counter = 0
for title, year in title_year_set:
    if counter < 4500:
        counter += 1
        continue
    if counter == 5000:
        break
    
    response_dict = dict()
    uri_title = quote(title)
    url = api_url.format(uri_title, year, api_key)
    response = requests.get(url)
    try:
        response_dict = json.loads(response.text)
    except:
        print('failed to parse for movie ', title)
        continue
    
    if response_dict['Response'] == 'False':
        prev_year = year - 1
        url = api_url.format(uri_title, prev_year, api_key)
        response = requests.get(url)
        try:
            response_dict = json.loads(response.text)
        except:
            print('failed to parse for movie ', title)
            continue
        response_dict = json.loads(response.text)
    
    response_dict['Title'] = title
    response_data.append(response_dict)
    counter += 1
    
response_data"""

In [8]:
#imdb_df = pd.DataFrame(response_data)
#imdb_df.to_csv('imdb_budget.csv', mode='a', header=False)

In [ ]:
"""import pandas as pd
from urllib.parse import quote
import requests
import json

response_data = []
title_year_set = set()

api_key = '6d4eee8d'
api_url = 'http://www.omdbapi.com/?t={}&y={}&apikey={}'

failed_df = pd.read_csv('failed_downloads.csv')
failed_df

titles_and_years = failed_df[['title', 'year']]
for index, row in titles_and_years.iterrows():
    title_year_set.add((row['title'], row['year']))

    
title_year_set = sorted(title_year_set)
counter = 0
for title, year in title_year_set:
    if counter < 0:
        counter += 1
        continue
    if counter == 500:
        break
    
    response_dict = dict()
    uri_title = quote(title)
    url = api_url.format(uri_title, year, api_key)
    response = requests.get(url)
    try:
        response_dict = json.loads(response.text)
    except:
        print('failed to parse for movie ', title)
        continue
    
    if response_dict['Response'] == 'False':
        prev_year = year - 1
        url = api_url.format(uri_title, prev_year, api_key)
        response = requests.get(url)
        try:
            response_dict = json.loads(response.text)
        except:
            print('failed to parse for movie ', title)
            continue
        response_dict = json.loads(response.text)
    
    response_dict['Title'] = title
    response_data.append(response_dict)
    counter += 1
    
response_data"""

In [119]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

imdb_budget_df = pd.read_csv('imdb_budget.csv')

imdb_budget_df['Actors'] = imdb_budget_df['Actors'].astype('category')
reduced_imdb = imdb_budget_df.loc[imdb_budget_df['Response'] == True]
reduced_imdb = imdb_budget_df.loc[imdb_budget_df['Type'] == 'movie']
mask = reduced_imdb['Actors'].notnull()
reduced_imdb = reduced_imdb.dropna(subset=['Actors'])
ohe_actors = reduced_imdb.Actors.str.get_dummies(sep=',').add_prefix('actor_')

In [121]:
ohe_genres = reduced_imdb.Genre.str.get_dummies(sep=',').add_prefix('genre_')

In [ ]:
import re

def paranthesis_remover(entry):
    return re.sub('\(.+?\)', '', str(entry))

reduced_imdb['Writer'] = reduced_imdb['Writer'].apply(paranthesis_remover)

ohe_writers = reduced_imdb.Writer.str.get_dummies(sep=',').add_prefix('writer_')
ohe_writers

In [ ]:
reduced_imdb['Director'] = reduced_imdb['Director'].apply(paranthesis_remover)

ohe_directors = reduced_imdb.Director.str.get_dummies(sep=',').add_prefix('director_')
ohe_directors

In [124]:
ohe_languages = reduced_imdb.Language.str.get_dummies(sep=',').add_prefix('language_')

In [127]:
reduced_imdb = reduced_imdb.join(ohe_languages, lsuffix='caller', rsuffix='other')

In [ ]:
reduced_imdb = reduced_imdb.join(ohe_actors, lsuffix='caller', rsuffix='other')
reduced_imdb = reduced_imdb.join(ohe_directors, lsuffix='caller', rsuffix='other')
reduced_imdb = reduced_imdb.join(ohe_genres, lsuffix='caller', rsuffix='other')
reduced_imdb = reduced_imdb.join(ohe_writers, lsuffix='caller', rsuffix='other')

In [ ]:
reduced_imdb = reduced_imdb.drop(columns=['Actors', 'Language', 'Writer', 'Genre', 'Director'])

In [140]:
reduced_imdb.drop(columns=['Unnamed: 0'], inplace=True)

In [198]:
reduced_imdb.drop(columns=['Awards', 'Plot', 'Error', 'Poster'], inplace=True)